In [24]:
import LiveTrading
import asyncio
import nest_asyncio
import TradeClient
import MarketDataFetcher
import time
from pprint import pprint

nest_asyncio.apply()

# 클라이언트
c_obj = TradeClient.FuturesOrder()

# 마켓 
m_obj = MarketDataFetcher.FuturesMarket()

# 라이브트레이드
t_obj = LiveTrading.LiveTradingManager(seed_money=10, market='futures', requested_leverage=5, kline_period=2, stop_loss_rate=0.35, init_stop_rate=0.015)
asyncio.run(t_obj.start_update_account())

dummy_signal = (True, 2, 1)

symbol = 'AGLDUSDT'

async def balance_print():
    available_ = asyncio.run(c_obj.get_available_balance())
    total_ = asyncio.run(c_obj.get_total_wallet_balance())
    print(f'거래가능 예수금 : {available_}')
    print(f'총 평가금액    : {total_}')
    return (available_, total_)

ava, ttl = asyncio.run(balance_print())
price = float(asyncio.run(m_obj.fetch_symbol_price(symbol))['price'])
timestamp = int(time.time() * 1000)
t_obj.ins_portfolio.update_log_data(symbol=symbol, price=price, timestamp=timestamp)
log_ = t_obj.ins_portfolio.data_container.get_data(f'futures_{symbol}')
# pprint(log_)

seed money 조정 : 10.00 -> 7.69
거래가능 예수금 : 0.05564748
총 평가금액    : 7.68916386


In [25]:
log_.stop_price

1.922820356

In [28]:
(1.36 / 7)*100

19.42857142857143

In [ ]:
price = float(asyncio.run(m_obj.fetch_symbol_price(symbol))['price'])
timestamp = int(time.time() * 1000)
t_obj.ins_portfolio.update_log_data(symbol=f'futures_{symbol}', price=price, timestamp=timestamp)
close_signal = asyncio.run(t_obj.submit_order_close_signal(symbol))

In [ ]:
print(asyncio.run(c_obj.get_total_wallet_balance()))
print(t_obj.ins_portfolio.total_balance)
pprint(t_obj.ins_portfolio.__dict__)
      

a = 0.123456789

In [ ]:
a = 0.123456789
print(round(a, 8))

In [ ]:
asyncio.run(c_obj.get_available_balance())

In [ ]:
asyncio.run(c_obj.fetch_order_history('XRPUSDT'))[-1]

In [ ]:
(2.7*2.52680) * 0.99

In [ ]:
pprint(t_obj.ins_portfolio.trade_history)

In [38]:
a ={"a":123}

In [ ]:
if a:
    print(True)

In [11]:
data = t_obj.ins_portfolio.data_container.get_data(data_name='123')

AttributeError: No attribute named '123'